# 第二十章 迭代和解析

列表解析: 用于把任意一个表达式而不是一个函数应用于一个迭代对象中的元素, 比 map 和 filter 更加通用

__列表解析与 map__

In [1]:
res = list(map(ord,'spam'))
res

[115, 112, 97, 109]

列表解析用于在一个序列上应用一个任意的表达式,将其结果收集到一个新的列表中并返回

In [ ]:
res = [ord(x) for x in 'spam']
res

[115, 112, 97, 109]

In [ ]:
list(map((lambda x: x**2), range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

__列表解析增加循环和嵌套__

In [ ]:
[x for x in range(10) if x % 2 ==0]

[0, 2, 4, 6, 8]

In [ ]:
list(filter((lambda x: x % 2 == 0), range(10)))

[0, 2, 4, 6, 8]

In [ ]:
res = [x+y for x in range(3) if x % 2==0 for y in range(3) if y%2==0]
res

[0, 2, 2, 4]

__列表解析和矩阵__

In [ ]:
M = [
    [1,2,3],
    [4,5,6],
    [7,8,9]
]
N = [
    [2,2,2],
    [3,3,3],
    [4,4,4],
]

In [ ]:
M[1]

[4, 5, 6]

In [ ]:
M[1][2]

6

In [ ]:
[row[0] for row in M]

[1, 4, 7]

### 理解列表解析

对于现在运行的 Python 下的测试, map 调用比等效的 for 循环快两倍,列表解析比 map 循环更加快. 主要是因为 map 和 列表是在解释器中以 C 语言的速度来运行的

1. for 循环可以让逻辑变得更加清晰,建议使用
2. map 和列表解析也是比较容易理解的,并且能够改善应用运行的速度

In [ ]:
open('data.x').readlines()

['aa\n', 'bbhello \n', 'world \n']

In [ ]:
[line.strip() for line in open('data.x')]

['aa', 'bbhello', 'world']

In [ ]:
[line.rstrip() for line in open('data.x')]

['aa', 'bbhello', 'world']

In [ ]:
list(map((lambda line: line.strip()), open('data.x')))

['aa', 'bbhello', 'world']

## 生成器

Python 对于延迟提供了更多的支持 --- 它提供了工具在需要的时候才产生结果,而不是立即产生结果. 可以通过两种语言结构延迟结果创建

* 生成器函数: 编写为常规的 def 语句,但是使用 yield 语句一次返回一个结果,在每个结果之间挂起和继续他们的状态
* 生成器表达式: 返回按照需要产生结果的一个对象, 而不是构建一个结果列表

### 生成器函数 yield VS return
* 生成器函数: 返回一个值并且从随后退出的地方继续的函数. 生成器函数是随着时间产生的一个序列
* 创建时自动实现迭代器协议

__状态挂起__
* 生成器函数自动在生成值的时刻挂起并继续函数的执行. 通常用于提起计算一系列值以及在类中手动保存和恢复状态都很有用
* 生成器函数在挂起时保存的状态包含他们的整个本地作用域, 当函数恢复时, 他们的本地变量保存了信息并且使得其可用
* yield 语句挂起该函数并且像调用者返回一个值,但是保留足够的状态使得函数能够从他们离开的地方继续. 
* 需要继续时, 函数在上一个 yield 返回后立即继续执行

__迭代协议整合__

对于生成器协议的支持:
1. 函数需要包含一条 yield 语句, 进行调用时, 他们返回一个迭代器对象, 该对象支持用一个名为__next__ 的自动创建的方法来继续执行的接口
2. 生成器的 return 位于 def 语句块的末尾, 用于终止值的生成

__生成器函数__

In [1]:
def gen_squares(N):
    for i in range(N):
        yield i**2
    return

In [3]:
for i in gen_squares(5):
    print(i, end=':')

0:1:4:9:16:

In [4]:
x = gen_squares(4)

In [5]:
x

<generator object gen_squares at 0x1038acf90>

In [6]:
next(x)

0

In [7]:
next(x)

1

生成器在内存使用和性能方面都更好. 允许函数避免临时在做所有的工作. 当结果的列表很大或者在处理每一个结果都需要很多的时间时, 非常有用

## 生成器表达式
* 生成器表达式就像一般的列表解析一样, 他们是在圆括号中,而不是方括号中的
* 生成器表达式不在内存中构建结果,而是返回一个生成器对象, 该对象支持迭代器协议


In [9]:
G = (x ** 2 for x in range(4))

In [10]:
next(G)

0

In [11]:
next(G)

1

In [12]:
next(G)

4

生成器表达式可以对于内存空间进行优化,但是不一定对于性能能够进行优化

In [15]:
G = (c*4 for c in 'spam')
list(G)

['ssss', 'pppp', 'aaaa', 'mmmm']

In [16]:
def time_four(s):
    for c in s:
        yield c * 4

In [17]:
list(time_four('spam'))

['ssss', 'pppp', 'aaaa', 'mmmm']

生成器表达式和生成器函数自身都是迭代器,但是只支持一次活跃迭代. 一旦任何迭代器运行到完成, 所有的迭代器都将耗尽

### 迭代工具模拟 zip 和 map


模拟 map 函数

In [18]:
def my_map(func, *seqs):
    res = []
    for args in zip(*seqs):
        res.append(func(*args))
    
    return res

In [19]:
print(my_map(abs, [0,-1,2,-3]))

[0, 1, 2, 3]


In [20]:
print(my_map(pow, [1,2,3,4],[3,4,5,6,]))

[1, 16, 243, 4096]


使用列表解析可以更简单的实现上述功能

In [21]:
def my_map(func, *seqs):

    return (func(*args) for args in zip(*seqs))

In [24]:
print(
    my_map(
        pow,
        [1, 2, 3, 4],
        [
            3,
            4,
            5,
            6,
        ],
    )
)

[1, 16, 243, 4096]


实现自己的 zip 和 map

不带有默认版本的 zip

In [2]:
def my_zip(*seqs, pad=None):
    seqs = [list(item) for item in seqs]
    res = []
    while any(seqs):
        res.append(tuple((seq.pop(0) if seq else pad) for seq in seqs))

    return res

In [3]:
s1 = "abc"
s2 = "xyz123"

print(my_zip(s1, s2,pad=99))

[('a', 'x'), ('b', 'y'), ('c', 'z'), (99, '1'), (99, '2'), (99, '3')]


生成器版本的 zip

In [4]:
def my_zip(*seqs, pad=None):
    seqs = [list(item) for item in seqs]
    while any(seqs):
        yield tuple(seq.pop(0) if seq else pad for seq in seqs)

In [6]:
s1 = "abc"
s2 = "xyz123"

print(list(my_zip(s1, s2, pad=99)))

[('a', 'x'), ('b', 'y'), ('c', 'z'), (99, '1'), (99, '2'), (99, '3')]


In [35]:
import time
import sys
reps = 10000
repslist= range(reps)

def timer(func, *args,**kwargs):
    start = time.time()
    for _ in repslist:
        ret = func(*args,**kwargs)
    elapsed = time.time() - start

    return elapsed,ret

In [37]:
def for_loop():
    res = []
    for x in repslist:
        res.append(abs(x))
    
    return res

def list_comp():
    return [abs(x) for x in repslist]

def map_call():
    
    return list(map(abs, repslist))

def gen_expr():
    return list(abs(x) for x in repslist)

def gen_func():
    def gen():
        for x in repslist:
            yield abs(x)
    
    return list(gen())    


开始记录时间

In [38]:
print(sys.version)

3.10.7 (v3.10.7:6cc6b13308, Sep  5 2022, 14:02:52) [Clang 13.0.0 (clang-1300.0.29.30)]


In [39]:
for test in [for_loop, list_comp,map_call, gen_expr,gen_func]:
    elapsed,result = timer(test)
    print('-'*33)
    print("%-9s: %.5f => [%s...%s]" % (test.__name__, elapsed, result[0],result[-1]))

---------------------------------
for_loop : 4.50421 => [0...9999]
---------------------------------
list_comp: 2.83421 => [0...9999]
---------------------------------
map_call : 1.60770 => [0...9999]
---------------------------------
gen_expr : 4.03837 => [0...9999]
---------------------------------
gen_func : 4.01984 => [0...9999]


# 第二十一章: 模块

## import 是如何工作的

Python 中的 import 并非是把一个文件插入到另一个文件中. 导入其实是运行时的运算, 程序第一次导入指定文件时,会执行三个步骤:
1. 找到模块文件
2. 编译成位码
3. 执行模块的代码来创建其所定义的对象

1. __搜索__: Python 必须查找 import 语句所引用的模块文件. Python 使用了标准模块搜索路径来查找 import 语句所对应的模块文件
2. __编译__: 如果有需要会编译为字节码, 如果存在字节码,直接加载字节码. 如果发现字节码的文件比源代码的文件就,就会重新编译字节码
3. __运行__: 依次执行文件中所有语句的字节码文件

### 搜索
1. 程序的主目录. 找到即停止, 先搜索先使用
2. PYTHONPATH 目录(如果进行了设置)
3. 任何.pth 文件的内容 (如果存在该文件)
4. 标准库链接目录

已经导入的模块查看

In [43]:
list(sys.modules.keys())

['sys',
 'builtins',
 '_frozen_importlib',
 '_imp',
 '_thread',
 '_warnings',
 '_weakref',
 '_io',
 'marshal',
 'posix',
 '_frozen_importlib_external',
 'time',
 'zipimport',
 '_codecs',
 'codecs',
 'encodings.aliases',
 'encodings',
 'encodings.utf_8',
 '_signal',
 '_abc',
 'abc',
 'io',
 '__main__',
 '_stat',
 'stat',
 '_collections_abc',
 'genericpath',
 'posixpath',
 'os.path',
 'os',
 '_sitebuiltins',
 '__future__',
 'itertools',
 'keyword',
 '_operator',
 'operator',
 'reprlib',
 '_collections',
 'collections',
 'types',
 '_functools',
 'functools',
 'contextlib',
 '_virtualenv',
 '_distutils_hack',
 'site',
 'importlib._bootstrap',
 'importlib._bootstrap_external',
 'warnings',
 'importlib',
 'importlib.machinery',
 'importlib._abc',
 'importlib.util',
 'runpy',
 'enum',
 '_sre',
 'sre_constants',
 'sre_parse',
 'sre_compile',
 '_locale',
 'copyreg',
 're',
 'fnmatch',
 'ntpath',
 'errno',
 'urllib',
 'urllib.parse',
 'pathlib',
 'collections.abc',
 'typing.io',
 'typing.re',
 '

#### 配置搜索路径


查看平台配置的模块搜索路径; python 会从左到右自动搜索该文件的路径

In [1]:
import sys
print(sys.path)

['/Users/lixun1/PycharmProjects/DatascientificProject/notebooks', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/lixun1/PycharmProjects/DatascientificProject/venv/lib/python3.10/site-packages']


默认的 sys 目录添加:
1. 顶级文件的主目录
2. 任何 PYTHONPATH 目录
3. 已经创建的包含 .pth 文件路径的内容
4. 标准库目录

1. 通过修改 sys.path 进行导入路径的临时性修改
2. 通过 PYTHONPATH 和 .pth 用于提供路径的持久化修改

模块的处理函数:
1. import: 使用一个变量名引用整个模块变量, 需要通过模块名来获取对象的属性
2. from: 将变量名复制到该作用域, 建议直接使用该变量名
3. from *: 将所有变量名引用到该作用域
4. impl.reload(): 不终止程序的情况下,提供了一种重新载入模块文件代码的方法

导入只有在第一次import 或 from 时,载入并且执行,并且只执行一次

import 和 from 是可执行的语句,而不是编译期间的声明, 可以嵌套在 if 测试中,出现在函数 def 之中
1. import 将整个模块的对象赋值给一个变量名
2. from 用于将一个或者多个变量名赋值给另一个模块中同名的对象

reload:
1. reload 是 Python 中的内置函数, 而不是语句
2. 传给 reload 的是已经存在的模块对象, 而不是变量名
3. 

__init__.py: 包导入语句的路径中的每个目录内,都必须有 __init__.py 文件,否则包导入会失败. 
1. dir1 和 dir2 中必须有一个 __init__py 文件
2. dir0 是容器,不需要 __init__.py

1. 包的初始化: Python 首次导入某个目录时, 会自动执行该目录下 __init__.py 文件中的所有程序代码. 包含创建数据文件 连接数据库
2. 模块命名空间的初始化: 在包导入的模型中, 目录内的目录路径, 在导入后会变成真实的嵌套对象路径. 
3. from 语句的行为: 可以在 __init__.py 文件内使用  __all__ 列表来定义

# 第二十四章 在模块中隐藏数据

* 在变量名前防止 _x 可以用于防止客户端使用 from * 语句导入模块时, 把其中的那些变量名复制出去
* 在模块顶层把变量名的字符串列表赋值给变量 __all__, 用于达到类似 _x 命名惯例的隐藏效果
  __all__ = ['Error', 'encode','decode'] 用于指出需要复制的变量名

* __name__ : 如果文件是以顶层程序文件运行的话, 在启动时, __name__ 就会被设置为字符串 __main__ 
* 如果文件被导入, __name__ 就会改成客户端所了解的模块名
* 模块可以检测自己的 __name__, 来确定他是在执行还是在导入
* __name__ 可以用来进行最简单的单元测试

In [5]:
def minmax(test, *args):
    res = args[0]
    for arg in args[1:]:
        if test(arg,res):
            res = arg

    return res

def lessthan(x,y): return x < y

def grtrthan(x,y): return x > y


if __name__=='__main__':
    print(minmax(lessthan, 4,3,2,1,4,5,6,3))
    print(minmax(grtrthan, 4,3,2,1,4,5,6,3))    
    

1
6


## 修改模块的搜索路径

sys.path 在程序启动时进行初始化，并在此之后，可以任意对其元素进行删除，附加和重设.

In [7]:
import sys
sys.path

['/Users/lixun1/PycharmProjects/DatascientificProject/notebooks',
 '/opt/anaconda3/lib/python311.zip',
 '/opt/anaconda3/lib/python3.11',
 '/opt/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/opt/anaconda3/lib/python3.11/site-packages',
 '/opt/anaconda3/lib/python3.11/site-packages/aeosa']

In [8]:
sys.path.append('')

## 模块是对象： 元程序
模块通过内置属性显示了他们大多数的有趣的属性，因此可以很容易的编写程序来管理其他程序。这类程序称为元程序，在其他系统之上工作，也称为内省。程序可以看见可处理对象的内部

* M.name
* M.__dict__['name']
* sys.modules['M'].name
* getattr(M,'name')

# 第二十五章: OOP 宏伟蓝图

* OOP 提供了一种不同寻常的而且往往是更有效的检查程序的方式,利用这种设计方法,分解代码,降低代码的冗余度,通过定制现有的代码来编写新的程序,而不是在原处修改
* 类的设计可以用来创建和管理新的对象
* 类的两个作用:
* * 继承: 继承父类的属性,并且扩展子类的属性
* * 组合: 通过组合其他对象实例,来完成相应的指令,定义自己的行为以及类关系
* * 类是对象的工厂: 类基本上就是产生对象的工厂, 每次调用一个类, 就会产生一个有独立命名空间的新对象. 每个通过类产生的对象都可以读取类的属性,并获得自己的命名空间来存储数据
* * 通过继承进行定制: 支持在类的外部重新定义其属性,从而扩充该类.
* * 运算符重载: 类可以定义对象来响应在内置类型上的几种运算. 切片 级联和索引等运算. 通过类钩子可以用来试下任何内置类型的运算


## 概览 OOP



* 像函数那样调用类会创建新的实例对象
* 每个实例对象继承类的属性并且获得了自己命名空间
* 在方法内对 self 属性做赋值运算会产生每个实例自己的属性

In [1]:
class FirstClass:
    def setdata(self, value):
        self.data = value
    def display(self):
        print(self.data)


In [2]:
x = FirstClass()
y = FirstClass()

In [3]:
x.setdata("King Arthur")
y.setdata(3.21156)

In [4]:
x.display()

King Arthur


In [5]:
y.display()

3.21156


In [6]:
x.data = "New Value"
x.display()

New Value


In [7]:
x.anothername = "name"

In [8]:
x.anothername

'name'

* 超类列在了类开头的括号中
* 类从其超类中继承属性
* 实例会继承所有可读取类的属性
* 每个 object.attribute 都会开启新的独立的搜索
* 逻辑的修改是通过创建子类，而不是修改超类

In [9]:
class SecondClass(FirstClass):
    def display(self):
        print('Current value="%s"' % self.data)

In [10]:
z = SecondClass()
z.setdata(42)
z.display()

Current value="42"


* 类可以截获 Python 运算符
* 以 __x__ 双下换线命名的方法是特殊的钩子。Python 运算符重载的实现是提供特殊命名的方法来拦截运算。Python 语言替每种运算和特殊命名的方法之间，定义了固定不变的映射关系
* 当实例出现内置运算时，这些方法会自动调用
* 类可覆盖多数内置类型运算
* 运算符覆盖方法没有默认值
* 运算符可以让类与 Python 的对象模型相互集成

In [13]:
class ThirdClass(SecondClass):
    def __init__(self,value):
        self.data = value
    def __add__(self, other):
        return ThirdClass(self.data + other)
    def __str__(self):
        return '[ThirdClass: %s]' % self.data
    def mul(self,other):
        self.data *= other

In [14]:
a = ThirdClass('abc')
a.display()

Current value="abc"


In [15]:
print(a)

[ThirdClass: abc]


In [16]:
b = a + 'xyz'

In [17]:
print(b)

[ThirdClass: abcxyz]


In [18]:
a.mul(3)

In [19]:
print(a)

[ThirdClass: abcabcabc]


最简单的类

In [20]:
class rec:pass

In [22]:
rec.name = 'bob'
rec.age=24

In [23]:
print(rec.name)
print(rec.age)

bob
24


In [24]:
x = rec()
y = rec()

In [25]:
x.name,y.name

('bob', 'bob')

如果一个列没有实例化，为其动态添加的属性是存储在类空间的

In [26]:
x.name='Sun'

In [28]:
x.name,y.name,rec.name

('Sun', 'bob', 'bob')

命名空间对象的属性通常都是以字典的形式实现的，而类继承树只是连接到其他字典的字典而已

In [30]:
rec.__dict__.keys()

dict_keys(['__module__', '__dict__', '__weakref__', '__doc__', 'name', 'age'])

In [31]:
x.__dict__.keys()

dict_keys(['name'])

In [32]:
y.__dict__.keys()

dict_keys([])

每个实例都会连接到其类以便于进行继承，可以通过 ____class____ 进行查看

In [33]:
x.__class__

__main__.rec

类有一个__bases__属性，是其超类的元组

In [34]:
rec.__bases__

(object,)

In [36]:
x.__class__

__main__.rec

Python 的类和实例只是命名空间对象，属性是通过赋值语句动态建立
及时是方法也可以完全独立的在任意对象的外部创建

In [37]:
def uppperName(self):
    return self.name.upper()

In [38]:
uppperName(x)

'SUN'

将函数赋值给类，作为属性

In [39]:
rec.method = uppperName

In [41]:
x.method()

'SUN'

In [42]:
y.method()

'BOB'

In [44]:
rec.method(x)

'SUN'

#### 类与字典的关系 

In [45]:
rec = {}
rec['name'] = 'mel'
rec['age'] = 45
rec['job'] = 'trainer/writer'

In [46]:
print(rec['name'])

mel


通过类来记录参数的代价要比字典等价形式少很多。 通过空类来产生命名空间对象

In [47]:
class rec:pass

In [48]:
rec.name = 'mel'
rec.age = 45
rec.job = 'trainer/writer'

In [49]:
print(rec.age)

45


# 第二十七章

OOP 中的核心概念：
1. 创建实例：填充实例属性
2. 定义行为方法：在类方法中封装逻辑
3. 运算符重载：为打印行为提供内置的操作行为
4. 定制行为：重新定义子类中的方法使其特殊化
5. 定制构造函数：为超类步骤添加初始化逻辑

委托：委托是一种常用代码模式的代表，是一种复合的结构，管理一个包装的对象并且吧方法调用传递给他

# 第二十八章： 类编写细节


把简单的非函数的对象赋值给类属性，就会产生数据属性，并且由实例共享

In [1]:
class SharedData:
    spam = 42


In [3]:
x=SharedData()
y=SharedData()
x.spam,y.spam

(42, 42)

可以通过类名修改，通过实例或者类引用

In [4]:
SharedData.spam=99
x.spam,y.spam

(99, 99)

通过实例进行修改不会影响所有共享变量的值，而是创建对应的实例变量

In [6]:
x.spam=42
x.spam,y.spam,SharedData.spam

(42, 99, 99)

* 对于实例的属性进行赋值运算会在该实例内创建或者修改变量名，而不是在共享的类中
* 继承搜索只会发生在属性引用时，而不是在赋值运算时发生：对对象属性的赋值总是会修改该对象


In [14]:
class MixedNames:
    data = 'spam'
    def __init__(self,value):
        self.data = value
    def display(self):
        print(self.data,MixedNames.data)

通过构造函数进行修改

In [15]:
x = MixedNames(1)
y = MixedNames(2)

In [18]:
x.display();y.display()

1 spam
2 spam


将属性存储在不同的对象内，可以决定其可见范围；附加在类上时，变量名是共享的；附加在实例上时，变量名是属于每个实例的数据，非共享数据

## 方法
* Python 会自动将实例方法的调用对应到类方法函数；
* class 通过Python 继承搜索流程找出方法名称所在之处
* 通过 self 提供的钩子，返回调用的主体，本质上就是实例对象
* 在 class 语句的作用域中赋值的对象，就会变成类对象的属性，然后由这个类创建的每个实例继承

In [19]:
class NextClass:
    def printer(self,text):
        self.message = text
        print(self.message)

In [20]:
x = NextClass()
x.printer('instance call')

instance call


In [21]:
x.message

'instance call'

方法只能够通过实例或类本身两种两种方法中的任意一种执行; 通过累的形式进行传递有相同的效果，只需要在类形式中传递了相同的实例对象

In [25]:
NextClass.printer(x,'class call')

class call


In [26]:
x.message

'class call'

## 调用超类构造函数

方法一般是通过实例调用的，但是对于构造函数之类的属性__init__ 都是通过继承进行查找的，Python 找出并且只调用一个 __init__. 如果要保证子类的构造函数执行超类的构造逻辑，一般需要通过类明确的调用超类的__init__ 方法

In [37]:
class Super:
    def __init__(self,x):
        self.x = x
class Sub(Super):
    def __init__(self,x,y):
        # Super.__init__(self,x)
        super().__init__(x)
        self.y = y 
    

In [38]:
I = Sub(1,2)

In [39]:
I.x

1

In [40]:
I.y

2

* 静态方法：支持编写不预期第一参数为实例对象的方法，支持这类方法向无实例的函数那样运作
* 类方法：调用的时候接收一个类，而不是一个实例，可以用于管理一个类的数据

后续会补充

## 继承

class 语句这样的命名空间的重点就是支持变量名继承 \n 
在 Python 中，对于对象进行点运算符时，就会发生继承，而且涉及了搜索属性定义树（一个或者多个命名空间）。Python 会从头至尾搜索命名空间树，先从对象开始，寻找所能找到的第一个 attr

### 继承方法的专有化
* 系统设计：将系统设计成类的层次，在新增外部的子类对其进行扩展，而不是在原处修改已经存在得了逻辑
* 子类完全可以取代继承的属性，提供超类可以找到的属性，并且通过已经覆盖的方法回调超类来扩展超类的方法

In [42]:
class Super:
    def method(self):
        print('in Super.method')

class Sub(Super):
    def method(self):
        print('starting Sub.method')
        Super.method(self)
        print('ending Sub.method')

### 类接口技术

In [ ]:
class Super:
    def method(self):
        print("in Super.method")

    def delegate(self):
        self.action()


class Inheritor(Super):
    pass


class Extender(Super):
    def method(self):
        print("stating Extender.method")
        Super.method(self)
        print("ending Extender.method")


class Replacer(Super):
    def method(self):
        print("In Replace.method")


class Provider(Super):
    def action(self):
        print("in Provider.action")


if __name__ == "__main__":
    for klass in (Inheritor, Replacer, Extender):
        print("\n" + klass.__name__ + "...")
        klass().method()
        print("\n Provider...")
        x = Provider()
        x.delegate()

### 抽象超类

类的部分行为是由子类所提供的：
1. x.delete 调用中，首先按照继承向上进行查找，直到在 Super 中找到 delegate 方法，并且将 x 传递给这个方法的 self 参数
2. 在 Super.delegate 方法中，self.action 会对 self 以及在他上层的对象启动新的独立继承搜索，self 表示 Provider，并且在 Provider 子类中找到 action 方法

抽象基类在 Python3 中可以通过 @abstractmethod 装饰器进行创建

* 类：class 语句声明的赋值语句。类是产生对象的工厂
* 对象：由类产生的对象
* 实例：self 指的是就是 self 

## 命名空间

可以通过类读取其属性，但是无法从 def 语句外读取函数或方法内的局部变量。
局部变量只有在 def 内的其余代码才是可见的

#### 命名空间字典


* 模块的命名空间实际上是以字典的形式实现的，并且可以由内置属性 __dict__ 显示这一点；属性点运算符其实就是字典的索引运算，属性继承就是搜索链接的字典而已
* 实例和对象就是 Python 中带有链接的字典而已


In [2]:
class Super:
    def hello(self):
        self.data = 'spam'

In [3]:
class Sub(Super):
    def hola(self):
        self.data2 = 'eggs'

实例通过 __class__属性找到其类，类通过 __bases__ 属性找到其超类

In [5]:
x = Sub()

实例的命名空间字典

In [6]:
x.__dict__

{}

实例所属的类

In [7]:
x.__class__

__main__.Sub

类的父类

In [9]:
Sub.__bases__

(__main__.Super,)

In [10]:
Super.__bases__

(object,)

当类为 self 属性赋值时，会填入实例对象。属性最后位于实例的属性命名空间字典内，而不是类的。实例对象的命名空间保存了数据，会随着实例的不同而不同，self 是进入其命名空间的钩子

In [11]:
y = Sub()

In [12]:
x.hello()

In [13]:
x.__dict__

{'data': 'spam'}

In [14]:
x.hola()

In [15]:
x.__dict__

{'data': 'spam', 'data2': 'eggs'}

In [17]:
list(Sub.__dict__.keys())

['__module__', 'hola', '__doc__', '__annotations__']

In [18]:
Super.__dict__.keys()

dict_keys(['__module__', 'hello', '__dict__', '__weakref__', '__doc__'])

In [19]:
y.__dict__

{}

In [20]:
dir(Sub)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'hello',
 'hola']

In [21]:
dir(Super)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'hello']

In [22]:
dir(x)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data',
 'data2',
 'hello',
 'hola']

# 第二十九章 运算符重载

## 构造函数和表达式：__init__ 和 __sub__

In [35]:
class Number:
    def __init__(self,start):
        self.data = start
    def __sub__(self,other):
        return Number(self.data - other)


In [36]:
x = Number(5)

In [37]:
y = x - 2

In [38]:
y.data

3

## 索引和分片

In [42]:
class Indexer:
    def __getitem__(self, index):
        return index ** 2

In [43]:
x = Indexer()

In [44]:
x[2]

4

In [45]:
for i in range(5):
    print(x[i], end=' ')

0 1 4 9 16 

支持分片

In [46]:
class Indexer:
    data = [5,6,7,8,9]
    def __getitem__(self, index):
        print('getitem', index)
        return self.data[index]
    def __setitem__(self,index,value):
        self.data[index] = value

In [47]:
x = Indexer()

In [48]:
x[0]

getitem 0


5

In [49]:
x[1]

getitem 1


6

In [51]:
x[1:4]

getitem slice(1, 4, None)


[6, 7, 8]

In [52]:
x[-1]

getitem -1


9

通过 __setitem__ 索引赋值方法类似的拦截索引和分片赋值，为后者接受一个分片对象，以同样的方式传递到另一个索引赋值中

* 索引迭代：__getitem__
* 定义该方法可以实现for 循环每次循环时调用类的 __getitem__ ，并持续搭配有更高的偏移值；任何响应索引运算的内置活用户定义的对象，都会响应迭代


In [1]:
class stepper:
    def __getitem__(self,i):
        return self.data[i]

In [2]:
x = stepper()

In [3]:
x.data = 'spam'

In [4]:
for item in x:
    print(item, end=' ')

s p a m 

对于任何支持 for 循环的类都会自动支持 Python 所有的迭代环境；例如：成员关系测试 in、列表解析、内置函数 map、列表和元组赋值运算符以及类型构造方法

In [5]:
'p' in x

True

In [6]:
[c for c in x]

['s', 'p', 'a', 'm']

In [7]:
list(map(str.upper, x))

['S', 'P', 'A', 'M']

In [8]:
list(x)

['s', 'p', 'a', 'm']

In [9]:
tuple(x)

('s', 'p', 'a', 'm')

In [10]:
' '.join(x)

's p a m'

## 迭代器对象：__iter__ 和 __next__

* Python 中的迭代环境都会先尝试 __iter__ 方法，然后尝试 __getitem__ 方法。只有不支持迭代器协议时，才会尝试使用索引运算
* 优先使用 __iter__ 然后尝试使用__getitem__
* 迭代器是通过内置函数 iter 去尝试寻找 __iter__ 方法来实现的，而这种方法应该返回一个迭代器对象。如果已经提供了，Python 就会重复调用这个迭代器的 next 方法，直到触发 Stopiteration 异常；否则改用 __getiterm__ 机制，通过索引偏移量重复索引，直到触发 IndexError 异常


In [13]:
class Squares:
    def __init__(self,start,stop):
        self.value = start - 1
        self.stop = stop
    
    def __iter__(self):
        return self

    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
        self.value +=1
        return self.value ** 2

In [12]:
for i in Squares(1,5):
    print(i, end=' ')

1 4 9 16 25 

In [20]:
x = Squares(1,5)
i = iter(x)

In [21]:
next(i)

1

In [22]:
next(i)

4

__iter__ 机制是在__getitem__ 中所见到的其他所有迭代环境的实现方式（成员关系测试、类构造函数、序列赋值运算）

In [24]:
x = Squares(1,5)

In [25]:
[n for n in x]

[1, 4, 9, 16, 25]

In [26]:
[n for n in x]

[]

#### 多个迭代器对象
*  单迭代对象：生成器函数和表达式、map 以及 zip 内置函数
*  多迭代对象：range 函数、其他内置类型支持多个活跃的迭代器。如果需要实现多个迭代器，__iter__ 只需要定义新的迭代器对象，而不是返回 self

In [8]:
class SkipIterator:
    def __init__(self,wrapped):
        self.wrapped = wrapped
        self.offset = 0
    def __next__(self):
        if self.offset >= len(self.wrapped):
            raise StopIteration
        else:
            item = self.wrapped[self.offset]
            self.offset += 2
            return item

class SkipObject:
    def __init__(self,wrapped):
        self.wrapped =  wrapped
    def __iter__(self):
        return SkipIterator(self.wrapped)
if __name__ == "__main__":
    alpha = "abcedf"
    skipper = SkipObject(alpha)
    I = iter(skipper)
    

In [9]:
next(I),next(I),next(I)

('a', 'c', 'd')

In [10]:
for x in skipper:
    for y in skipper:
        print(x+y,end=' ')

aa ac ad ca cc cd da dc dd 

#### 成员关系：__contaions__、__iter__ 和 __getitem__

* 在迭代领域，类通常把 in 成员关系运算符实现为一个迭代，使用 __item__ 方法或者 __getitem__ 方法，如果要支持更加特定的成员关系，需要编写 __contains__ 方法；
* _contains_方法优先于：_iter_ 方法，__iter__ 方法优先于__getitem__ 方法
* _contains_ 用于将成员关系定义为一个映射应用键（可以快速查找），用于进行序列的搜索


In [11]:
class Iters:
    def __init__(self,value):
        self.data = value
    
    def __getitem__(self,i):
        print('get[%s]: ' % i, end=' ')
        return self.data[i]

    def __iter__(self):
        print('iter=> ', end=' ')
        self.ix = 0
        return self
    def __next__(self):
        print("next:", end='')
        if self.ix == len(self.data): raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item

    def __contains__(self,x):
        print('contains: ', end=' ')
        return x in self.data    

In [12]:
x = Iters([1,2,3,4,5])

In [13]:
print(3 in x)

contains:  True


In [14]:
for i in x:
    print(i, end=' | ')

iter=>  next:1 | next:2 | next:3 | next:4 | next:5 | next:

In [15]:
print([i**2 for i in x])

iter=>  next:next:next:next:next:next:[1, 4, 9, 16, 25]


In [16]:
print(list(map(bin,x)))

iter=>  next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']


In [17]:
i = iter(x)
while True:
    try:
        print(next(i),end='@')
    except StopIteration:
        break

iter=>  next:1@next:2@next:3@next:4@next:5@next:

分片

In [19]:
x = Iters('spam')
x[0]

get[0]:  

's'

## 属性调用：__getattr__ 和 __setattr__
* _getattr_ 放放是拦截属性点号运算符。对于未定义的属性名称和实例进行点运算时，就会用属性名称作为字符串调用者那个方法
* 如果 Python 可以通过继承树搜索到这个属性，则不会触发该方法的调用。_getattr_ 可以作为钩子来通过通用的方式来响应属性需求

In [23]:
class empty:
    def __getattr__(self,attrname):
        if attrname == "age":
            return 40
        else:
            raise AttributeError(attrname)

In [24]:
x = empty()

In [25]:
x.age

40

In [26]:
x.name

AttributeError: name

__setattr__ 会拦截所有属性的赋值语句。如果定义了这个方法：self.attr = value 就会变成 self.__setattr__('attr',value)；
在 __setattr__ 执行 self 赋值，都是导致再次调用__setattr__，导致无穷递归循环，需要通过属性字典进行索引计算进行赋值：self.__dict__['name'] = x
而不是self.name = x

In [28]:
class accesscontrol:
    def __setattr__(self,attr,value):
        if attr=='age':
            self.__dict__[attr] = value
        else:
            raise AttributeError(attr + ' not allowed')

In [29]:
x = accesscontrol()

In [30]:
x.age = 40

In [31]:
x.age

40

In [32]:
x.name = 'mel'

AttributeError: name not allowed

## 其他属性管理工具

* __getattribute__ 方法拦截所有的属性获取，不仅仅是那些没有定义的，使用时比习更加小心防止出现循环
* Property：内置函数允许我们把方法和特定雷属性上的获取和设置操作关联起来
* 描述符提供了一个协议，把一个类的__get__和__set__方法与特定类属性的方法关联起来

In [51]:
class PrivateExc(Exception): pass
class Privacy:
    def __setattr__(self,attrname,value):
        if attrname in self.privates:
            raise PrivateExc(attrname,self)
        else:
            self.__dict__[attrname] = value

class Test1(Privacy):
    privates = ['age']

class Test2(Privacy):
    privates = ['name','pay']
    def __init__(self):
        self.__dict__['name'] = 'Tom'

In [52]:
x = Test1()
y = Test2()

In [53]:
x.name = 'Bob'

In [54]:
y.name = 'Sue'

PrivateExc: ('name', <__main__.Test2 object at 0x11e103910>)

In [55]:
y.age = 30

In [56]:
x.age = 40

PrivateExc: ('age', <__main__.Test1 object at 0x11e102da0>)

通过上面的方式可以再 Python 中实现属性私有性的首选方法。更建议的方法是通过类装饰器来实现私有属性

#### __repr__ 和 __str__ 会返回字符串表达形式

当实例打印后者转换成字符串时，_repr_ 或者其近亲_str_ 就会自动调用

In [1]:
class adder:
    def __init__(self,value=0):
        self.data = value
    def __add__(self,other):
        self.data += other

In [2]:
x = adder()

In [3]:
print(x)

In [4]:
x

In [5]:
class addrer(adder):
    def __repr__(self):
        return 'addrrepr(%s) ' % self.data

In [6]:
x = addrer(2)
x + 1

In [7]:
x

addrrepr(3) 

In [8]:
print(x)

addrrepr(3) 


In [10]:
str(x),repr(x)

('addrrepr(3) ', 'addrrepr(3) ')

* 打印操作通常尝试_str_和 str 内置函数（print 运行的内部等价形式）。返回用户一个友好的提示
* __repr__ 用于所有其他的环境中：在交互式模式下，用于提示回应，以及 repr 函数，可以用于创建提示，或者给开发者一个更加详细的提示
* 在交互式模式中，只能够实用 __repr__

In [11]:
class addrstr(adder):
    def __str__(self):
        return '[value:%s ]' % self.data

In [12]:
x = addrstr(3)

In [13]:
x + 1

In [14]:
x

In [15]:
print(x)

[value:4 ]


In [16]:
str(x),repr(x)

('[value:4 ]', '<__main__.addrstr object at 0x108d87cd0>')

In [17]:
class addboth(adder):
    def __str__(self):
        return '[Value: %s]' % self.data
    def __repr__(self):
        return '[addboth(%s)]' % self.data

In [18]:
x = addboth(4)

In [19]:
x+1

In [20]:
x

[addboth(5)]

In [21]:
print(x)

[Value: 5]


* str 和 repr 都必须返回字符串，其他结果类型不会被转换，并且会引发错误
* __str__ 有好的用户提示只有当对象出现在一个打印操作顶层的时候才应用。嵌入到大对象中的对象可能用其__repr__ 或者默认的方法打印


In [1]:
class Printer:
    def __init__(self,val):
        self.val = val
    def __str__(self):
        return str(self.val)

In [3]:
objs = [Printer(2),Printer(3)]
for x in objs:
    print(x)

2
3


In [4]:
print(objs)

[<__main__.Printer object at 0x1116d2770>, <__main__.Printer object at 0x1116d2110>]


编写 repr 来适应第二种情况

In [5]:
class Printer:
    def __init__(self,val):
        self.val = val
    def __repr__(self):
        return str(self.val)

In [6]:
objs = [Printer(2),Printer(3)]
for x in objs:print(x)

2
3


In [7]:
print(objs)

[2, 3]


在打印并且可以看到定制显示的时候，肯定是使用了__str__或者__repr__中的任何一个

#### 右侧加法 radd 和 iadd

* 如果需要实现右侧使用实例对象的加法，需要增加__radd__方法。只有当+右侧的对象是类实例，而左边对象不是类实例的时候，会调用__radd__；
* 主要取决于类实例位于 + 左边还是右边


In [16]:
class Commuter:
    def __init__(self, val):
        self.val = val
    def __add__(self,other):
        print('add', self.val, other)
        return self.val + other
        
    def __radd__(self, other):
        print('radd', self.val, other)
        return other + self.val

In [17]:
x = Commuter(88)
y = Commuter(99)

In [18]:
x + 1

add 88 1


89

In [19]:
1 + y

radd 99 1


100

In [20]:
x + y

add 88 <__main__.Commuter object at 0x110c17f10>
radd 99 88


187

类型测试可能需要辨别它是否能够安全的转换并由此避免嵌套 ？？？？

#### 原处加法
为了实现 += 原处相加，编写 iadd 或者 add，如果 iadd 不存在，则使用 add

In [21]:
class Number:
    def __init__(self,val):
        self.val = val
    def __iadd__(self, other):
        self.val += other
        return self

In [22]:
x = Number(5)

In [23]:
x += 1

In [24]:
x+=1

In [25]:
x.val

7

使用 add 实现原处相加

In [30]:
class Number:
    def __init__(self,val):
        self.val =  val
    def __add__(self,other):
        return Number(self.val + other)

In [31]:
x = Number(5)

In [32]:
x += 1

In [33]:
x+=1

In [34]:
x.val

7

## Call 表达式：call

如果定义了 call 表达式，Python 就会为实例应用表达式运行__call__ 方法，让类实例的外观和用法类似于函数

In [42]:
class Callee:
    def __call__(self, *pargs,**kargs):
        print('Called:', pargs, kargs)

In [43]:
c = Callee()

In [44]:
c(1,2,3)

Called: (1, 2, 3) {}


* 带有一个__call__ 的类和实例，支持与常规函数和方法完全相同的参数语法和语义
* 拦截调用表达式允许类实例模拟类似函数的外观，但是也在掉用户总保持了状态信息以供使用


In [54]:
class Prod:
    def __init__(self,value):
        self.value = value
    def __call__(self,other):
        return self.value * other
    def comp(self,other):
        return self.value * other

In [55]:
x = Prod(2)

In [56]:
x(3)

6

In [57]:
x(4)

8

通过一个简单的方法也可以通过上面类似的功能

In [58]:
x.comp(3)

6

In [59]:
x.comp(4)

8

除了 init str repr, call 是最常用的重载方法了

#### 布尔测试： bool 和 len

Python 首先尝试 __bool__ 来获取一个直接的 bool 值，如果没有该方法，尝试根据 __len__ 根据对象的长度确定一个真值

In [60]:
class Truth:
    def __bool__(self):
        return True

In [61]:
x = Truth()

In [63]:
if x:print('yes!')

yes!


通过 len 判断长度是一个费控对象为真

In [64]:
class Truth:
    def __len__(self):
        return 0

In [65]:
x = Truth()

In [67]:
if not x:print('bool')

bool


如果都没有定义，则一定为真

In [68]:
class Truth:
    pass

In [69]:
x = Truth()

In [70]:
bool(x)

True

## 对象析构函数 del
产生实例时，调用 __init__ 构造函数，没当实例被回收时，执行析构函数 __del__ 

In [80]:
class Life:
    def __init__(self,name='unknown'):
        print("hello", name)
        self.name = name
    
    def __del__(self):
        print("GoodBye", self.name)

In [81]:
brain = Life('Brain')

hello Brain


1. 当 brain 赋值为字符串时，我们会失去 Life 实例的最后一个引用，因此出发析构函数，此时需要一些清理行为
2. Python 在实例回收时，会自动回收该实例所拥有的全部空间，对于空间管理不需要析构函数


In [82]:
brain = 'Loretta'

GoodBye Brain


# 第三十章：类的设计

In [ ]:
aa